# AI for BI

Generating a chart

We'll use CrewAI for the processing. This includes a Python interpreter which will be used to create the charts. You must have Docker installed locally to run the code automatically.

Running code generated by an LLM is potentially dangerous as there is n guarantee that the code wil be safe. Running the code in a Docker container, however, provides a sandbox for the code to run in and isolates it from the application code and local operating system.


Import the necessary libraries and set the LLM model (defaults to using OpenAI)

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
llm = "gpt-4o-mini"

Import the tools required to read and write files

In [3]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

Set up the ``chart_agent``

In [4]:
# Define agent

chart_agent = Agent(
        role="Chart creator",
        goal="""Read the data provided and create a chart from that data.
                If you are given specific instructions on how to draw the chart then follow them,
                if not then create a chart that best represents all of the data""",
        backstory="You aim is to read and analyse sales data and create a chart",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=True
    )



The following defines a function that will create a task to create a markdown summary and chart for each data file and set up a crew.

We define the files and then loop through them calling the ``create_charts`` function to write the individual markdown files.


In [5]:
files = [
    {
        'data_file_name':'sales_product_cat.csv',
        'chart_file_name': 'sales_product_summary.png',
        
    },
    {
        'data_file_name': 'monthly_sales.csv',
        'chart_file_name': 'monthly_sales.png',
    },
    {
        'data_file_name': 'sales_by_region.csv',
        'chart_file_name': 'sales_by_region.png',
    }
]



In [6]:
create_chart = Task(
    description="""Create a chart for {data_file_name} and save it in {chart_file_name}.'
                    """,
    expected_output="""A matplotlib chart""",
    agent=chart_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crew
crew = Crew(
    agents=[chart_agent],
    tasks=[create_chart],
    verbose=True
)
result = crew.kickoff_for_each(inputs=files)

2024-11-14 13:15:18,730 - 25692 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Chart creator
## Task: Create a chart for sales_product_cat.csv and save it in sales_product_summary.png.'
                    


# Agent: Chart creator
## Thought: I need to read the content of the file named 'sales_product_cat.csv' in order to create a chart that summarizes the sales data. After reading the data, I can determine the most appropriate type of chart to represent it.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"sales_product_cat.csv\"}"
## Tool Output: 
Product Category,Product Name,Units Sold,Revenue ($),% Change (Month-over-Month)
Electronics,Smart TV,1500,450000,+10%
Electronics,Laptop,1200,850000,+5%
Electronics,Smartphone,2000,1200000,-2%
Home Appliances,Refrigerator,700,400000,+8%
Home Appliances,Washing Machine,500,200000,-3%
Home Appliances,Microwave,900,150000,+15%
Furniture,Sofa,300,90000,+20%
Furniture,Dining Table,150,80000,+12%
Furniture,Bed Frame,250,130000,+7%



# Agent: Chart creator
## Thought: Thought: I have success

2024-11-14 13:19:55,173 - 25692 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Chart creator
## Task: Create a chart for monthly_sales.csv and save it in monthly_sales.png.'
                    


# Agent: Chart creator
## Thought: I need to read the content of the "monthly_sales.csv" file in order to create a chart and save it as "monthly_sales.png".
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"monthly_sales.csv\"}"
## Tool Output: 
Month,Total Units Sold,Total Revenue ($),% Change (Month-over-Month)
January,10000,3500000,-
February,10500,3675000,+5%
March,11000,3850000,+4.8%
April,10800,3780000,-1.8%
May,11500,4000000,+5.8%
June,11300,3950000,-1.3%
July,12000,4100000,+3.8%
August,11800,4000000,-2.4%



# Agent: Chart creator
## Thought: Thought: I have successfully read the content of the "monthly_sales.csv" file. It contains the sales data for each month, including total units sold and total revenue. Now, I will create a chart to visualize this data using Matplotlib and save it as "monthly_sales.png".
## Using tool: Code Int

2024-11-14 13:20:53,718 - 25692 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Chart creator
## Task: Create a chart for sales_by_region.csv and save it in sales_by_region.png.'
                    


# Agent: Chart creator
## Thought: I need to read the content of the "sales_by_region.csv" file to understand the sales data by region and then create a chart using Matplotlib. After that, I will save the chart as "sales_by_region.png".
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"sales_by_region.csv\"}"
## Tool Output: 
Region,Total Revenue ($),Top-Selling Product,Units Sold,% Change (Month-over-Month)
North America,1500000,Smartphone,800,+5%
Europe,1200000,Laptop,600,+8%
Asia-Pacific,800000,Smart TV,900,-2%
South America,350000,Refrigerator,300,+10%
Africa,250000,Sofa,150,+12%



# Agent: Chart creator
## Thought: Thought: I have read the sales data from the "sales_by_region.csv" file. The data contains information on regions, total revenue, top-selling products, units sold, and month-over-month changes. I will now create a bar 

In [7]:
data_analysis_agent = Agent(
        role="Data Analyser",
        goal="""You aim is to read and analyse sales data. You should
                then write a report on sales performance that includes an executive summary.
                """,
        backstory="You are assigned to perform sales analysis for a company",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=False
    )


In [10]:
write_report = Task(
    description=f"""The following contains a set of data files and corresponding charts:
                        {files}
                    Write report in Markdown that includes and overview of all of the sales data and incorporate 
                    the corresponding charts.

                    If the information is available, or you can calculate it,
                    try and answer the following questions: 
                    1. What has been the overall revenue for the latest month?
                    2. What are the top selling 5 items during the reporting period?
                    3. In which regions have there been the most sales and what items are popular those regions?
                    4. What sort of growth has there been over the reporting period?
                    5. Are there any trends that you can detect?

                    The overview of the data and the corresponding charts from {files} should be included in an appendix.
                    
                    Save the result in the file './report.md'.
                    """,
    expected_output="""A markdown file""",
    agent=data_analysis_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crewwrite_overview
crew = Crew(
    agents=[data_analysis_agent],
    tasks=[write_report],
    verbose=True
)
result2 = crew.kickoff()

2024-11-14 13:41:25,227 - 25692 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Data Analyser
## Task: The following contains a set of data files and corresponding charts:
                        [{'data_file_name': 'sales_product_cat.csv', 'chart_file_name': 'sales_product_summary.png'}, {'data_file_name': 'monthly_sales.csv', 'chart_file_name': 'monthly_sales.png'}, {'data_file_name': 'sales_by_region.csv', 'chart_file_name': 'sales_by_region.png'}]
                    Write report in Markdown that includes and overview of all of the sales data and incorporate 
                    the corresponding charts.

                    If the information is available, or you can calculate it,
                    try and answer the following questions: 
                    1. What has been the overall revenue for the latest month?
                    2. What are the top selling 5 items during the reporting period?
                    3. In which regions have there been the most sales and what items are popular those regions?
                    4. What sort of gr

![](image.png)